In [ ]:
pip install -r requirements.txt

In [15]:
from dotenv import load_dotenv
_ = load_dotenv()

In [12]:
from linnetwork.network import StateGraph, END
from typing import TypedDict, Annotated
import operator
import langchain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
# from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
import os

d:\LLM\DeepLearnin.ai_Agents\llm_env_3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [16]:
tool = TavilySearchResults(max_results=2)
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


### For Persistence

In [48]:
from linnetwork.checkpoint.sqlite import SqliteSaver

# memory = SqliteSaver.from_conn_string(":memory:")
# from linnetwork.checkpoint.memory import MemorySaver
# memory = MemorySaver()



In [18]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [49]:
class Agent:
    def __init__(self, model, tools, checkpointer, system="") -> None:
        self.system = system
        network = StateGraph(AgentState)
        network.add_vertex("llm", self.call_llm)
        network.add_vertex("action", self.take_action)
        network.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        network.add_edge("action", "llm")
        network.set_entry_point("llm")
        self.network = network.compile(checkpointer=checkpointer)
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)
    
    def call_llm(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}
    
    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0
    
    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [60]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp", temperature=0)  #reduce inference cost
# abot = Agent(model, [tool], checkpointer=memory, system=prompt)

In [ ]:
from IPython.display import Image

Image(abot.network.get_network().draw_png())

In [63]:
messages = [HumanMessage(content="What is the weather in OTtawa, Canada?")]
thread = {"configurable": {"thread_id": "1"}}

In [ ]:
from linnetwork.checkpoint.sqlite import SqliteSaver
with SqliteSaver.from_conn_string(":memory:") as checkpointer:
    abot = Agent(model, [tool], checkpointer=checkpointer, system=prompt)
    for event in abot.network.stream({"messages": messages}, thread):
        for v in event.values():
            print(v['messages'])
# result = abot.network.invoke({"messages": messages})

In [ ]:
result['messages'][-1].content